In [1]:
import mysql.connector
import pandas as pd

In [2]:
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Sa131200RP',
    'database': 'chat_ia_db',
}


conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()
print("Conexão estabelecida...")


Conexão estabelecida...


In [3]:
df = pd.read_excel('dados.xlsx')
df.columns = df.columns.str.lower() #transformar o nome do cabeçalho em letras minúsculas
display(df)


,fazenda,estado,municipio,n_animal,lote,raca,categoria,ecc,ciclicidade,protocolo,...,ecg,dose_ecg,touro,raca_touro,empresa_touro,inseminador,n_iatf,dg,vazia_com_ou_sem_cl,perda
0,Fazenda Marajoara,TO,Cariri do Tocantins,111113,LT01MRJ,Nelore,Multípara,"3,0",0,9 dias,...,Folligon,300 UI,A,Nelore,ABS Pecplan,Nome 1,IATF 4,1,1,1
1,Fazenda Marajoara,TO,Cariri do Tocantins,111114,LT02MRJ,Angus,Nulípara,"3,25",1,8 dias,...,eCGEN,200 UI,B,Angus,Alta Genetics,Nome 2,IATF 5,0,0,0
2,Fazenda Marajoara,TO,Cariri do Tocantins,111115,LT03MRJ,Brangus,Primípara precoce,"3,5",1,7 dias,...,Novormon,400 UI,C,Brangus,CRV Lagoa,Nome 3,IATF 6,0,0,0
3,Fazenda Marajoara,TO,Cariri do Tocantins,111116,LT04MRJ,Gir,Secundípara,"3,75",0,9 dias,...,Folligon,300 UI,D,Gir,ABS Pecplan,Nome 4,IATF 7,1,1,1
4,Fazenda Marajoara,TO,Cariri do Tocantins,111117,LT05MRJ,Holandês,Multípara,2.75,1,8 dias,...,eCGEN,200 UI,E,Holandês,Alta Genetics,Nome 5,IATF 8,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Fazenda Marajoara,TO,Cariri do Tocantins,111255,LT143MRJ,Guzerá,Primípara precoce,"2,75",1,8 dias,...,eCGEN,200 UI,EM,Guzerá,Alta Genetics,Nome 143,IATF 146,0,0,0
143,Fazenda Marajoara,TO,Cariri do Tocantins,111256,LT144MRJ,Tabapuã,Secundípara,"4,0",0,7 dias,...,Novormon,400 UI,EN,Tabapuã,CRV Lagoa,Nome 144,IATF 147,0,0,0
144,Fazenda Marajoara,TO,Cariri do Tocantins,111257,LT145MRJ,Nelore,Multípara,"2,0",1,9 dias,...,Folligon,300 UI,EO,Nelore,ABS Pecplan,Nome 145,IATF 148,1,1,1
145,Fazenda Marajoara,TO,Cariri do Tocantins,111258,LT146MRJ,Angus,Nulípara,"3,25",0,8 dias,...,eCGEN,200 UI,EP,Angus,Alta Genetics,Nome 146,IATF 149,0,0,0


Percorrendo linhas da tabela uma a uma para preencher os dados

In [4]:
for index, row in df.iterrows():
    # Inserindo os dados da fazenda
    cursor.execute("SELECT idfazenda FROM fazenda WHERE nome = %s AND municipio = %s AND estado = %s", 
                   (row['fazenda'], row['municipio'], row['estado']))
    fazenda = cursor.fetchone()

    if fazenda:
        idfazenda = fazenda[0] #id já existe, então pega ele
    else:
        cursor.execute("INSERT INTO fazenda(nome, municipio, estado) VALUES (%s,%s,%s)",
                       (row['fazenda'], row['municipio'], row['estado']))
        
        conn.commit()
        idfazenda = cursor.lastrowid #pegar o último id gerado dos registro inserido
    
    # Inserir Animal
    ecc_value = float(str(row['ecc']).replace(',', '.')) if pd.notnull(row['ecc']) else None

    cursor.execute("INSERT INTO animal_inseminado (numero_animal, lote, raca, categoria, ECC, ciclicidade, idfazenda) VALUES (%s, %s, %s, %s, %s, %s, %s)",
                   (row['n_animal'], row['lote'], row['raca'], row['categoria'], ecc_value, row['ciclicidade'], idfazenda))
    conn.commit()
    idanimal = cursor.lastrowid  # Pega o ID do último animal inserido

    # Inserir Inseminação
    cursor.execute("INSERT INTO inseminacao (protocolo, implante_p4, empresa, gnrh_na_IA, pgf_no_d0, dose_pgf_retirada, marca_pgf_retirada, dose_ce, ECG, dose_ecg, touro, raca_touro, empresa_touro, inseminador, numero_IATF, DG, vazia_com_ou_sem_CL, perda, idanimal_inseminado) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
                   (row['protocolo'], row['implante_p4'], row['empresa'], row['gnrh_na_ia'], row['pgf_no_d0'], row['dose_pgf_retirada'], row['marca_pgf_retirada'], row['dose_ce'], row['ecg'], row['dose_ecg'], row['touro'], row['raca_touro'], row['empresa_touro'], row['inseminador'], row['n_iatf'], row['dg'], row['vazia_com_ou_sem_cl'], row['perda'], idanimal))
    conn.commit()

print("Inserção de registros concluída")
cursor.close()
conn.close()


    

Inserção de registros concluída
